## Assignment 2


### Cassandra Python

#### Importing libraries


In [ ]:
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra cluster

In [ ]:
clstr = Cluster()
session = clstr.connect()

### Use session to 'talk' to cassandra

In [ ]:
session.execute("DROP KEYSPACE IF EXISTS m14")

In [ ]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

In [ ]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS m14 WITH REPLICATION = {
        'class':'SimpleStrategy', 
        'replication_factor':3
    }
""")

In [ ]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

In [ ]:
session.execute("""
CREATE TABLE IF NOT EXISTS m14.inventory (
    SKU INT, 
    name TEXT, 
    description TEXT, 
    warehouse_num INT,
    PRIMARY KEY(SKU, warehouse_num)
);
""")

#### Loading the data

In [ ]:
df = pd.read_csv('m14data.csv')
df

In [ ]:
df.columns

In [ ]:
for index, row in df.iterrows():
    print(f"SKU = {row['SKU']}, name = {row['name']}, description = {row['description']}, warehouse_num = {row['warehouse_num']}")


In [ ]:
for index, row in df.iterrows():
    # Properly format and escape the string values
    formatted_name = str(row['name']).replace("'", "''")  # replace single quote with two single quotes for SQL
    formatted_description = str(row['description']).replace("'", "''")

    # Use the formatted strings in the SQL statement
    sql_statement = f"""
        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES ({row['SKU']}, '{formatted_name}', '{formatted_description}', {row['warehouse_num']});
        """

    print(sql_statement)  # For debugging
    session.execute(sql_statement)


In [ ]:
rows = session.execute("select (SKU, name, description, warehouse_num) from m14.inventory")
for row in rows:
    print(f"SKU={row[0][0]}, name={row[0][1]}, description={row[0][2]}, warehouse_num={row[0][3]}")


In [ ]:
rows = session.executrows = session.execute("SELECT (SKU, name, description, warehouse_num) FROM m14.inventory WHERE warehouse_num = 3 ALLOW FILTERING")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}, {row[0][3]}")